In [1]:
import os
import logging
import xlwings as xw
import pandas as pd
from time import sleep

#设置工作目录
main_path = r'D:\JupyterLabFiles\Futures\Futures_exchange_data\DCE'
os.chdir(main_path)

#设置日志记录
logging.basicConfig(filename='DCE_combine.log', level=logging.INFO, encoding='utf-8', 
                    format='%(asctime)s - %(levelname)s - %(message)s')

#获取工作列表脚本
def get_file_list(main_path):
    files_path = []
    for root, dirs, files in os.walk(main_path):
        for file in files:
            if file.endswith(('.csv', '.xls', '.xlsx')) and not file.startswith('~$'):
                files_path.append(os.path.join(root, file))
    return files_path
    
#处理文件脚本
def process_file(file):
    #AKSHARE的期货列名对应关系字典
    futures_column_mapping = {
    "合约" : "symbol",
    "日期" : "date",
    "开盘价" : "open",
    "最高价" : "high",
    "最低价" : "low",
    "收盘价" : "close",
    "成交量" : "volume",
    "持仓量" : "open_interest",
    "成交金额" : "turnover",
    "结算价" : "settle",
    "前结算" : "pre_settle",
    }
    #保存的期权列名
    options_column_mapping ={
    "商品名称" : "future_name",
    "合约名称" : "option_name",
    "交易日期" : "trade_date",
    "开盘价" : "open",
    "最高价" : "high",
    "最低价" : "low",
    "收盘价" : "close",
    "前结算" : "pre_settle",
    "结算价" : "settle",
    "DELTA" : "delta",
    "成交量" : "volume",
    "持仓量" : "open_interest",
    "成交额（万元）" : "trnover",
    "行权量" : "exercise_vol"
    }
    #使用xlwings读取文件，用DataFrame载入
    try:
        wb = xw.Book(file)
        sht = wb.sheets[0]
        table = sht.range(sht.used_range).value
        df = pd.DataFrame(table,columns=table[0])
        df = df.drop([0])
        wb.close()
        sleep(0.1)
        #标准化列名
        df = df.rename(columns = {"成交额":"成交金额","前结算价":"前结算"})
        #期权数据的处理
        if 'DELTA' in df.columns:
            #保留必要的列
            options_columns_to_keep = list(options_column_mapping.keys())
            df = df[options_columns_to_keep]
            #更改列名
            df = df.rename(columns = options_column_mapping)
            print(f'文件{file}处理完成！')
            return df, 'option'
        #期货数据的处理
        else:
            #保留必要的列
            futures_columns_to_keep = list(futures_column_mapping.keys())
            df = df[futures_columns_to_keep]
            #更改列名
            df = df.rename(columns = futures_column_mapping)
            #创建Variety列
            df['variety'] = df['symbol'].str.extract(r'([a-zA-Z]+)')
            print(f'文件{file}处理完成!')
            return df, 'future'
    except Exception as e:
        print(f'文件{file}出错{e}')
        return None

        
#运行程序
files = get_file_list(main_path)
datas = [process_file(i) for i in files]
futures = [i[0] for i in datas if i[1] == 'future']
options = [i[0] for i in datas if i[1] == 'option']
option_combined = pd.concat(options, ignore_index=True)
option_combined.to_csv('dce_options.csv', index=False, encoding='gbk')
print('期权数据 dce_options.csv 已保存!')
future_combined = pd.concat(futures, ignore_index=True)
future_combined.to_csv('dce_futures.csv', index=False, encoding='gbk')
print('期货数据 dce_futures.csv 已保存')

KeyboardInterrupt: 